In [1]:
from pathways.scenarios import run_scenarios, load_scenario_table
from pathways.simulation import load_configuration
from pathways.outputs import save_scenario_result_to_pandas

In [2]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

%matplotlib inline

In [5]:
from pathlib import Path
datadir = Path("tests/test_summaries")
resultsdir = "H:/Shared drives/APHIS  Projects/Pathways/Results/use_cases/"
basic_config = load_configuration(datadir / "config.yml")
rate_scenario_table = load_scenario_table(datadir / "contamination_rate_estimation.csv")
inspection_scenario_table = load_scenario_table(datadir / "inspection_scenarios.csv")
consignment_scenario_table = load_scenario_table(datadir / "consignment_scenarios.csv")

## Use Case 1: Estimate contamination rates from high quality inspection data

When inspection data obtained with known statistically valid inspection methods are available, the simulation can be used to estimate the consignment contamination rates by recreating the inspections and calibrating the contamination configuration until similar inspection outcomes are achieved.

In the example below, data from AQIM inspections of cut flower consignments are used to estimate possible contamination rate probability distributions.

In [6]:
num_consignments = 3313
fitted_contamination_rate_results = run_scenarios(
    config=basic_config,
    scenario_table=rate_scenario_table[0:4],
    seed=42,
    num_simulations=10,
    num_consignments=num_consignments,
    detailed=False,
)

Running scenario: rate scenario 1 box
Running scenario: rate scenario 1 stem random
Running scenario: rate scenario 1 stem clustered random
Running scenario: rate scenario 1 stem clustered continuous


In [7]:
df_fitted = save_scenario_result_to_pandas(
    fitted_contamination_rate_results,
    config_columns=[
        "name",
        "consignment name",
        "inspection name",
        "contamination/contamination_rate/parameters",
        "contamination/arrangement",
        "contamination/clustered/distribution",
        "contamination/clustered/contaminated_units_per_cluster",
    ],
    result_columns=[
        "true_contamination_rate",
        "false_neg",
        "intercepted",
        "total_missed_contaminants",
        "total_intercepted_contaminants",
        "avg_boxes_opened_completion",
    ],
)

In [8]:
df_fitted['consignment failure rate'] = df_fitted["intercepted"] / num_consignments

In [9]:
# Format dataframe
column_names = ["consignment name", "inspection name", "beta parameters", "contaminant arrangement", "cluster distribution", "infested boxes per cluster", "simulated contamination rate (mean)", "consignment failure rate"]
df_contamination_pretty = df_fitted.iloc[:,[1,2,3,4,5,6,7,13]]

df_contamination_pretty.columns = column_names
df_contamination_pretty.iloc[:,6] = df_contamination_pretty.iloc[:,6].round(decimals=4)
df_contamination_pretty.iloc[:,7] = df_contamination_pretty.iloc[:,7].round(decimals=4)
df_contamination_pretty
#df_contamination_pretty.to_csv(resultsdir + "contamination_rate_results.csv")

C:\Users\Owner\Anaconda3\envs\pandemic\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,consignment name,inspection name,beta parameters,contaminant arrangement,cluster distribution,infested boxes per cluster,simulated contamination rate (mean),consignment failure rate
0,mean 0.0027 sd 0.028 box,box hypergeometric 0.1 random,"[0.0093, 3.4253]",random,,,0.0025,0.0448
1,mean 0.0027 sd 0.028 item random,box hypergeometric 0.1 random,"[0.0093, 3.4253]",random,,,0.0029,0.0649
2,mean 0.0027 sd 0.028 item clustered random,box hypergeometric 0.1 random,"[0.0093, 3.4253]",clustered,random,50,0.0027,0.0502
3,mean 0.0027 sd 0.028 item clustered cont,box hypergeometric 0.1 random,"[0.0093, 3.4253]",clustered,continuous,50,0.0027,0.0502


In [ ]:
# If loading results from saved csv, uncomment and run this chunk.

#df_contamination_pretty = pd.read_csv(resultsdir + "contamination_rate_results.csv")

## Use Case 2: Measure the effect of deviations from sampling protocols

We then use the calibrated contamination rate distribution with mean 0.0019 and standard deviation 0.01875 (mid variability scenario) with a clustered contaminant arrangement to run sampling scenarios with fixed consignment assumptions. The outcomes of these scenarios provide information about the relative impacts of changes to inspection protocols. 

In [ ]:
# Hiding very long output for this cell (many printed messages related to clusters)
#%%capture capt
num_consignments = 4672
inspection_scenario_results = run_scenarios(
    config=basic_config,
    scenario_table=inspection_scenario_table,
    seed=42,
    num_simulations=1,
    num_consignments=num_consignments,
    detailed=False,
)
# uncomment to print output if desired
#capt.show()

In [ ]:
df_inspections = save_scenario_result_to_pandas(
    inspection_scenario_results,
    config_columns=[
        "name",
        "inspection/unit",
        "inspection/sample_strategy",
        "inspection/proportion/value",
        "inspection/hypergeometric/detection_level",
        "inspection/selection_strategy",
        "inspection/cluster/cluster_selection",

    ],
    result_columns=[
        "true_contamination_rate",
        "max_missed_contamination_rate",
        "avg_missed_contamination_rate",
        "max_intercepted_contamination_rate",
        "avg_intercepted_contamination_rate",
        "avg_boxes_opened_completion",
        "avg_boxes_opened_detection",
        "avg_items_inspected_completion",
        "avg_items_inspected_detection",
        "false_neg",
        "intercepted",
        "total_missed_contaminants",
        "total_intercepted_contaminants",
    ],
)

In [ ]:
df_inspections['consignment failure rate'] = df_inspections["intercepted"] / num_consignments
contaminated_consignments = df_inspections["false_neg"] + df_inspections["intercepted"]
df_inspections["inspection success rate"] = df_inspections["intercepted"] / contaminated_consignments
df_inspections["% missed contaminants"] = df_inspections["total_missed_contaminants"] / (df_inspections["total_missed_contaminants"] + df_inspections["total_intercepted_contaminants"])

In [ ]:
# Format dataframe
column_names = ["name", "inspection unit", "sample strategy", "sample parameter", "selection strategy", "cluster selection", "avg contamination rate", "max missed contamination rate", "avg missed contamination rate", "max intercepted contamination rate", "avg intercepted contamination rate", "boxes opened completion", "boxes opened detection", "items inspected completion", "items inspected detection", "missed", "intercepted", "missed contaminants", "intercepted contamininants", "consignment failure rate", "inspection success rate", "% missed contaminants", "sample size method", "selection method"]


In [ ]:

df_inspections_pretty = df_inspections.loc[:, df_inspections.columns != 'inspection/hypergeometric/detection_level']
hypergeometric_parameters = df_inspections.iloc[[0,1,2,3,4,5,6,7,12,13,14,15],4]
df_inspections_pretty.iloc[[0,1,2,3,4,5,6,7,12,13,14,15],3] = hypergeometric_parameters
df_inspections_pretty.iloc[:,3] = df_inspections_pretty.iloc[:,3].astype(str)

df_inspections_pretty["sample size method"] = df_inspections_pretty[['inspection/sample_strategy', 'inspection/proportion/value']].agg(' '.join, axis=1)
df_inspections_pretty["selection method"] = df_inspections_pretty[["inspection/unit", 'inspection/selection_strategy', 'inspection/cluster/cluster_selection']].agg(' '.join, axis=1)

df_inspections_pretty.columns = column_names
df_inspections_pretty.iloc[:,6:11] = df_inspections_pretty.iloc[:,6:11].round(decimals=4)
df_inspections_pretty.iloc[:,11:19] = df_inspections_pretty.iloc[:,11:19].astype(int)
df_inspections_pretty.iloc[:,19:22] = df_inspections_pretty.iloc[:,19:22].round(decimals=4)
df_inspections_pretty.iloc[:18,[0,1,21,22,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]]
df_inspections_pretty

#df_inspections_pretty.to_csv(resultsdir + "inspection_scenario_results.csv")

In [ ]:
# If loading results from saved csv, uncomment and run this chunk.

#df_inspections_pretty = pd.read_csv(resultsdir + "inspection_scenario_results.csv")

In [ ]:
colors = {"hypergeometric 0.1": "#1f78b4", "hypergeometric 0.05": "#a6cee3", "proportion 0.02":"#b2df8a"}
patch_1 = mpatches.Patch(color="#a6cee3", label="hypergeometric 0.05")
patch_2 = mpatches.Patch(color="#1f78b4", label="hypergeometric 0.1")
patch_3 = mpatches.Patch(color="#b2df8a", label="proportion 0.02")

In [ ]:
plt.figure(figsize=(18, 18), dpi=150)
plt.subplot(221)
plt.subplots_adjust(wspace=0.65,left=0.15,right=0.97, top=0.95, bottom=0.05)
plt.barh(df_inspections_pretty["name"], df_inspections_pretty["inspection success rate"], color=df_inspections_pretty['sample size method'].replace(colors))
plt.title("Inspection Success Rate", fontsize=24)
plt.ylabel("inspection method", fontsize=20)
plt.xlabel("rate", fontsize=20)
plt.yticks(ticks=np.arange(18),labels=df_inspections_pretty["selection method"], fontsize=20)
plt.xticks(fontsize=18)
plt.subplot(222)
plt.barh(df_inspections_pretty["name"], df_inspections_pretty["avg missed contamination rate"], color=df_inspections_pretty['sample size method'].replace(colors))
plt.title("Avg. Missed Contamination Rate", fontsize=24)
plt.xlabel("rates", fontsize=20)
plt.yticks(ticks=np.arange(18),labels=df_inspections_pretty["selection method"], fontsize=20)
plt.xticks(ticks=[0, 0.005,0.01,0.015,0.02,0.025], fontsize=18)

plt.subplot(223)
plt.subplots_adjust(wspace=0.65,left=0.22,right=0.95)
plt.barh(df_inspections_pretty["name"], df_inspections_pretty["boxes opened completion"], color=df_inspections_pretty['sample size method'].replace(colors))
plt.title("Boxes Opened", fontsize=24)
plt.ylabel("inspection method", fontsize=20)
plt.xlabel("boxes", fontsize=20)
plt.yticks(ticks=np.arange(18),labels=df_inspections_pretty["selection method"], fontsize=20)
plt.xticks(fontsize=18)
plt.subplot(224)
plt.barh(df_inspections_pretty["name"], df_inspections_pretty["items inspected completion"], color=df_inspections_pretty['sample size method'].replace(colors))
plt.title("Items Inspected", fontsize=24)
plt.xlabel("items", fontsize=20)
plt.legend(handles=[patch_1,patch_2,patch_3], loc = "lower right", fontsize=20)
plt.yticks(ticks=np.arange(18),labels=df_inspections_pretty["selection method"], fontsize=20)
plt.xticks(ticks=[0,1000,2000,3000,4000],fontsize=18)
plt.savefig(resultsdir + "inspection_scenario_plots.png")
plt.show()



##  Use Case 3: Measure the effect of changes in consignment characteristics

We then use the simulation with fixed inspection assumptions to answer questions about how inspection outcomes change with changes in consignments. Using the AQIM inspection protocol (box unit, hypergeometric sample with 0.1 detection level and 0.95 confidence level, random selection), we simulated multiple scenarios to reflect the following consignment scenarios:

* Larger boxes to represent maritime cargo (700 items/box)
* Small boxes in small consignments to represent direct-to-consumer consignments
* Less variability in contamination rates (lower standard deviation)
* Contamination using items versus boxes
* Random contaminant arrangement
* Various clustered contaminant arrangements (small, large, or continuous clusters)

In [ ]:
num_consignments = 4672
consignment_scenario_results = run_scenarios(
    config=basic_config,
    scenario_table=consignment_scenario_table,
    seed=42,
    num_simulations=1,
    num_consignments=num_consignments,
    detailed=False,
)

In [ ]:
df_consignments = save_scenario_result_to_pandas(
    consignment_scenario_results,
    config_columns=[
        "name",
        "consignment name",
        "consignment/boxes/min",
        "consignment/boxes/max",
        "consignment/items_per_box/default",
        "contamination/contamination_unit",
        "contamination/contamination_rate/distribution",
        "contamination/contamination_rate/parameters",
        "contamination/arrangement",
        "contamination/clustered/distribution",
        "contamination/clustered/contaminated_units_per_cluster",
        "contamination/clustered/random/cluster_item_width",
    ],
    result_columns=[
        "true_contamination_rate",
        "max_missed_contamination_rate",
        "avg_missed_contamination_rate",
        "max_intercepted_contamination_rate",
        "avg_intercepted_contamination_rate",
        "avg_boxes_opened_completion",
        "pct_boxes_opened_completion",
        "avg_boxes_opened_detection",
        "pct_boxes_opened_detection",
        "avg_items_inspected_completion",
        "pct_items_inspected_completion",
        "avg_items_inspected_detection",
        "pct_items_inspected_detection",
        "false_neg",
        "intercepted",
        "total_missed_contaminants",
        "total_intercepted_contaminants",
    ],
)

In [ ]:
df_consignments['consignment failure rate'] = df_consignments["intercepted"] / num_consignments
contaminated_consignments = df_consignments["false_neg"] + df_consignments["intercepted"]
df_consignments["inspection success rate"] = df_consignments["intercepted"] / contaminated_consignments
df_consignments["contaminated_consignments"] = contaminated_consignments
df_consignments["% missed contaminants"] = df_consignments["total_missed_contaminants"] / (df_consignments["total_missed_contaminants"] + df_consignments["total_intercepted_contaminants"])

In [ ]:
# Format dataframe 
column_names = ["name", "consignment name", "items per box", "contamination unit", "contamination parameters", "contaminant arrangement", "cluster distribution", "contaminated units per cluster", "cluster width", "avg contamination rate", "avg missed contamination rate", "avg intercepted contamination rate", "avg boxes opened per inspection", "pct box opened per simulation", "avg items inspected per inspection", "pct items inspected per simulation", "missed contaminants", "intercepted contamininants", "inspection success rate", "contaminated_consignments", "% missed contaminants"]
df_consignments_pretty = df_consignments

In [ ]:
df_consignments_pretty.iloc[:,12:17] = df_consignments_pretty.iloc[:,12:17].round(decimals=4)
df_consignments_pretty.iloc[:,17:29] = df_consignments_pretty.iloc[:,17:29].astype(int)
df_consignments_pretty.iloc[:,29:31] = df_consignments_pretty.iloc[:,29:31].round(decimals=4)
df_consignments_pretty.iloc[:,[31]] = df_consignments_pretty.iloc[:,[31]].astype(int)
df_consignments_pretty.iloc[:,[32]] = df_consignments_pretty.iloc[:,[32]].round(decimals=4)
df_consignments_pretty = df_consignments_pretty.iloc[:,[0,1,4,5,7,8,9,10,11,12,14,16,17,18,21,22,27,28,30,31,32]]
df_consignments_pretty.columns = column_names
df_consignments_pretty

#df_consignments_pretty.to_csv(resultsdir + "consignment_scenario_results.csv")

In [ ]:
# If loading results from saved csv, uncomment and run this chunk.

#df_consignments_pretty = pd.read_csv(resultsdir + "consignment_scenario_results.csv")

In [ ]:
df_cargo_config = df_consignments_pretty.loc[0:2,:]
df_contamination_rate_scenarios = df_consignments_pretty.loc[3:8,:]
df_contaminant_arrangement_scenarios = df_consignments_pretty.loc[9:15,:]

In [ ]:
plt.figure(figsize=(16, 4.8), dpi=150)
plt.subplot(221)
plt.subplots_adjust(wspace=0.6,left=0.18,right=0.99,top=0.9, hspace=1.2, bottom=0.2)
plt.barh(df_cargo_config["name"], df_cargo_config["inspection success rate"], color="#1f78b4")
plt.title("Inspection Success Rate", fontsize=24)
plt.xlabel("rate", fontsize=20)
plt.yticks(ticks=np.arange(3),labels=df_cargo_config["consignment name"], fontsize=20)
plt.xticks(fontsize=18)
plt.subplot(222)
plt.barh(df_cargo_config["name"], df_cargo_config["avg missed contamination rate"], color="#1f78b4")
plt.title("Avg. Missed Contamination Rate", fontsize=24)
plt.xlabel("rate", fontsize=20)
plt.yticks(ticks=np.arange(3),labels=df_cargo_config["consignment name"],fontsize=20)
plt.xticks(ticks=[0,0.001,0.002,0.003,0.004], fontsize=18)

plt.subplot(223)
plt.barh(df_cargo_config["name"], df_cargo_config["avg boxes opened per inspection"], color="#1f78b4")
plt.title("Boxes Opened per Consignment", fontsize=24)
plt.xlabel("boxes", fontsize=20)
plt.yticks(ticks=np.arange(3),labels=df_cargo_config["consignment name"], fontsize=20)
plt.xticks(fontsize=18)
plt.subplot(224)
plt.barh(df_cargo_config["name"], df_cargo_config["avg items inspected per inspection"], color="#1f78b4")
plt.title("Items Inspected per Consignment", fontsize=24)
plt.xlabel("items", fontsize=20)
plt.yticks(ticks=np.arange(3),labels=df_cargo_config["consignment name"], fontsize=20)
plt.xticks(fontsize=18)
#plt.savefig(resultsdir + "cargo_config_scenario_plots.png")
plt.show()

In [ ]:
plt.figure(figsize=(18.2, 7.5), dpi=150)
plt.subplot(221)
plt.subplots_adjust(wspace=0.6,left=0.18,right=0.97,bottom=0.1, hspace=0.6)
plt.barh(df_contamination_rate_scenarios["name"], df_contamination_rate_scenarios["inspection success rate"])
plt.title("Inspection Success Rate", fontsize=24)
plt.xlabel("rate", fontsize=20)
plt.yticks(ticks=np.arange(6),labels=df_contamination_rate_scenarios["consignment name"], fontsize=20)
plt.xticks(fontsize=18)
plt.subplot(222)
plt.barh(df_contamination_rate_scenarios["name"], df_contamination_rate_scenarios["avg missed contamination rate"])
plt.title("Avg. Missed Contamination Rate", fontsize=24)
plt.xlabel("rate", fontsize=20)
plt.yticks(ticks=np.arange(6),labels=df_contamination_rate_scenarios["consignment name"],fontsize=20)
plt.xticks(ticks=[0,0.002,0.004,0.006], fontsize=18)

plt.subplot(223)
plt.barh(df_contamination_rate_scenarios["name"], df_contamination_rate_scenarios["avg boxes opened per inspection"], color="#1f78b4")
plt.title("Boxes Opened per Consignment", fontsize=24)
plt.xlabel("boxes", fontsize=20)
plt.yticks(ticks=np.arange(6),labels=df_contamination_rate_scenarios["consignment name"], fontsize=20)
plt.xticks(fontsize=18)
plt.subplot(224)
plt.barh(df_contamination_rate_scenarios["name"], df_contamination_rate_scenarios["avg items inspected per inspection"], color="#1f78b4")
plt.title("Items Inspected per Consignment", fontsize=24)
plt.xlabel("items", fontsize=20)
plt.yticks(ticks=np.arange(6),labels=df_contamination_rate_scenarios["consignment name"], fontsize=20)
plt.xticks(fontsize=18)
#plt.savefig(resultsdir + "cargo_config_scenario_plots.png")
plt.show()



In [ ]:
plt.figure(figsize=(18, 8), dpi=150)
plt.subplot(221)
plt.subplots_adjust(wspace=0.86,left=0.22,right=0.95, bottom=0.1,hspace=0.5)
plt.barh(df_contaminant_arrangement_scenarios["name"], df_contaminant_arrangement_scenarios["inspection success rate"])
plt.title("Inspection Success Rate", fontsize=24)
plt.xlabel("rate", fontsize=20)
plt.yticks(ticks=np.arange(7),labels=df_contaminant_arrangement_scenarios["consignment name"], fontsize=20)
plt.xticks(fontsize=18)
plt.subplot(222)
plt.barh(df_contaminant_arrangement_scenarios["name"], df_contaminant_arrangement_scenarios["avg missed contamination rate"])
plt.title("Avg. Missed Contamination Rate", fontsize=24)
plt.xlabel("rate", fontsize=20)
plt.yticks(ticks=np.arange(7),labels=df_contaminant_arrangement_scenarios["consignment name"],fontsize=20)
plt.xticks(ticks=[0,0.002,0.004,0.006], fontsize=18)


plt.subplot(223)
plt.barh(df_contaminant_arrangement_scenarios["name"], df_contaminant_arrangement_scenarios["avg missed contamination rate"])
plt.title("Avg Missed Contamination Rate", fontsize=24)
plt.xlabel("rate", fontsize=20)
plt.yticks(ticks=np.arange(7),labels=df_contaminant_arrangement_scenarios["consignment name"], fontsize=20)
plt.xticks(ticks=[0, 0.002,0.004,0.006], fontsize=18)
plt.subplot(224)
plt.barh(df_contaminant_arrangement_scenarios["name"], df_contaminant_arrangement_scenarios["avg intercepted contamination rate"])
plt.title("Avg Intercepted Contamination Rate", fontsize=24)
plt.xlabel("rate", fontsize=20)
plt.yticks(ticks=np.arange(7),labels=df_contaminant_arrangement_scenarios["consignment name"], fontsize=20)
plt.xticks(ticks=[0,0.01,0.02,0.03,0.04], fontsize=18)
plt.savefig(resultsdir + "contaminant_arrangement_scenarios_plots.png")
plt.show()



In [ ]:
num_consignments = 25
air_scenario_results = run_scenarios(
    config=basic_config,
    scenario_table=consignment_scenario_table[0:1],
    seed=42,
    num_simulations=20,
    num_consignments=num_consignments,
    detailed=False,
)

In [ ]:
num_consignments = 14
maritime_scenario_results = run_scenarios(
    config=basic_config,
    scenario_table=consignment_scenario_table[1:2],
    seed=42,
    num_simulations=20,
    num_consignments=num_consignments,
    detailed=False,
)

In [ ]:
num_consignments = 2000
dtc_scenario_results = run_scenarios(
    config=basic_config,
    scenario_table=consignment_scenario_table[2:3],
    seed=42,
    num_simulations=20,
    num_consignments=num_consignments,
    detailed=False,
)

In [ ]:
df_consignments_1M = save_scenario_result_to_pandas(
    air_scenario_results+maritime_scenario_results+dtc_scenario_results,
    config_columns=[
        "name",
        "consignment name",
        "consignment/boxes/min",
        "consignment/boxes/max",
        "consignment/items_per_box/default",
        "contamination/contamination_unit",
        "contamination/contamination_rate/distribution",
        "contamination/contamination_rate/parameters",
        "contamination/arrangement",
        "contamination/clustered/distribution",
        "contamination/clustered/contaminated_units_per_cluster",
        "contamination/clustered/random/cluster_item_width",
    ],
    result_columns=[
        "true_contamination_rate",
        "max_missed_contamination_rate",
        "avg_missed_contamination_rate",
        "max_intercepted_contamination_rate",
        "avg_intercepted_contamination_rate",
        "avg_boxes_opened_completion",
        "pct_boxes_opened_completion",
        "avg_boxes_opened_detection",
        "pct_boxes_opened_detection",
        "avg_items_inspected_completion",
        "pct_items_inspected_completion",
        "avg_items_inspected_detection",
        "pct_items_inspected_detection",
        "false_neg",
        "intercepted",
        "total_missed_contaminants",
        "total_intercepted_contaminants",
    ],
)

In [ ]:
df_consignments_1M['consignment failure rate'] = df_consignments_1M["intercepted"] / num_consignments
contaminated_consignments = df_consignments_1M["false_neg"] + df_consignments_1M["intercepted"]
df_consignments_1M["inspection success rate"] = df_consignments_1M["intercepted"] / contaminated_consignments
df_consignments_1M["contaminated_consignments"] = contaminated_consignments
df_consignments_1M["% missed contaminants"] = df_consignments_1M["total_missed_contaminants"] / (df_consignments_1M["total_missed_contaminants"] + df_consignments_1M["total_intercepted_contaminants"])

In [ ]:
# Format dataframe 
column_names = ["name", "consignment name", "items per box", "contamination unit", "contamination parameters", "contaminant arrangement", "cluster distribution", "contaminated units per cluster", "cluster width", "avg contamination rate", "avg missed contamination rate", "avg intercepted contamination rate", "avg boxes opened per inspection", "pct boxes opened per simulation", "avg items inspected per inspection", "pct items inspected per simulation", "missed contaminants", "intercepted contamininants", "inspection success rate", "contaminated_consignments", "% missed contaminants"]
df_consignments_pretty_1M = df_consignments_1M

In [ ]:
df_consignments_pretty_1M.iloc[:,12:33] = df_consignments_pretty_1M.iloc[:,12:33].round(decimals=3)
#df_consignments_pretty_1M.iloc[:,17:29] = df_consignments_pretty_1M.iloc[:,17:29].astype(int)
#df_consignments_pretty_1M.iloc[:,29:33] = df_consignments_pretty_1M.iloc[:,29:33].round(decimals=4)
#df_consignments_pretty_1M.iloc[:,[31]] = df_consignments_pretty_1M.iloc[:,[31]].astype(int)
#df_consignments_pretty_1M.iloc[:,[32]] = df_consignments_pretty_1M.iloc[:,[32]].round(decimals=4)
df_consignments_pretty_1M = df_consignments_pretty_1M.iloc[:,[0,1,4,5,7,8,9,10,11,12,14,16,17,18,21,22,27,28,30,31,32]]
df_consignments_pretty_1M.columns = column_names
df_consignments_pretty_1M

#df_consignments_pretty_1M.to_csv(resultsdir + "consignment_scenario_1M_results.csv")

In [ ]:
plt.figure(figsize=(16, 7), dpi=150)
plt.subplot(321)
plt.subplots_adjust(wspace=0.6,left=0.18,right=0.99,top=0.9, hspace=1.2, bottom=0.2)
plt.barh(df_consignments_pretty_1M["name"], df_consignments_pretty_1M["inspection success rate"], color="#1f78b4")
plt.title("Inspection Success Rate", fontsize=24)
plt.xlabel("rate", fontsize=20)
plt.yticks(ticks=np.arange(3),labels=df_consignments_pretty_1M["consignment name"], fontsize=20)
plt.xticks(fontsize=18)
plt.subplot(322)
plt.barh(df_consignments_pretty_1M["name"], df_consignments_pretty_1M["avg missed contamination rate"], color="#1f78b4")
plt.title("Avg. Missed Contamination Rate", fontsize=24)
plt.xlabel("rate", fontsize=20)
plt.yticks(ticks=np.arange(3),labels=df_consignments_pretty_1M["consignment name"],fontsize=20)
plt.xticks(ticks=[0,0.001,0.002,0.003,0.004], fontsize=18)

plt.subplot(323)
plt.barh(df_consignments_pretty_1M["name"], df_consignments_pretty_1M["avg boxes opened per inspection"], color="#1f78b4")
plt.title("Boxes Opened per Consignment", fontsize=24)
plt.xlabel("boxes", fontsize=20)
plt.yticks(ticks=np.arange(3),labels=df_consignments_pretty_1M["consignment name"], fontsize=20)
plt.xticks(fontsize=18)
plt.subplot(324)
plt.barh(df_consignments_pretty_1M["name"], df_consignments_pretty_1M["avg items inspected per inspection"], color="#1f78b4")
plt.title("Items Inspected per Consignment", fontsize=24)
plt.xlabel("items", fontsize=20)
plt.yticks(ticks=np.arange(3),labels=df_consignments_pretty_1M["consignment name"], fontsize=20)
plt.xticks(ticks=[0,5000,10000,15000], fontsize=18)
#plt.savefig(resultsdir + "cargo_config_scenario_1M_plots.png")
#plt.show()

plt.subplot(325)
plt.barh(df_consignments_pretty_1M["name"], df_consignments_pretty_1M["pct boxes opened per simulation"], color="#1f78b4")
plt.title("% Boxes Opened per Simulation", fontsize=24)
plt.xlabel("% boxes", fontsize=20)
plt.yticks(ticks=np.arange(3),labels=df_consignments_pretty_1M["consignment name"], fontsize=20)
plt.xticks(fontsize=18)
plt.subplot(326)
plt.barh(df_consignments_pretty_1M["name"], df_consignments_pretty_1M["pct items inspected per simulation"], color="#1f78b4")
plt.title("% Items Inspected per Simulation", fontsize=24)
plt.xlabel("% items", fontsize=20)
plt.yticks(ticks=np.arange(3),labels=df_consignments_pretty_1M["consignment name"], fontsize=20)
plt.xticks(fontsize=18)
plt.savefig(resultsdir + "cargo_config_scenario_1M_plots_fixed.png")
plt.show()